In [1]:
# scoring as up to the day of 2017-10-01
# Track the sales from 2017-10-01 to 2018-09-30
# Need to rerun

import pandas as pd
import numpy as np
import datetime
import os
import logging
import glob
import gc
pd.options.display.max_columns=99
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Loyalty_Health'

In [2]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

In [3]:
lastdate_date=datetime.date(2017,9,30) # Last day
active_Sunday=str(lastdate_date-datetime.timedelta(days=52*7-1))

lapsed_Sunday=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))
Beginning_18_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))

lastdate=str(lastdate_date)
print("Lapsed Start on: "+lapsed_Sunday) #>=
print("Active Start on: "+active_Sunday) #>=
print("Store Allocation Starting on: "+Beginning_18_months_ago) #>=
print("Last Saturday: "+lastdate) #<=

Lapsed Start on: 2016-04-03
Active Start on: 2016-10-02
Store Allocation Starting on: 2016-04-03
Last Saturday: 2017-09-30


In [4]:
historical_subclass_files=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")
historical_subclass_files=[x for x in historical_subclass_files if x.split("ediaStormDailySales_week_ending_")[1][:10]<="2017-09-30"]

historical_subclass_files.sort()

print(min([x.split("ediaStormDailySales_week_ending_")[1][:10] for x in historical_subclass_files]))
print(max([x.split("ediaStormDailySales_week_ending_")[1][:10] for x in historical_subclass_files]))


2016-07-02
2017-09-30


In [5]:
# 1 out of the 2
list_df_transactions_18_months=[]
i_counter=0
for file in historical_subclass_files:
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
        df=df.rename(columns={"item_transaction_amt":"sales"})
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','subclass_transaction_amt'])
        df=df.rename(columns={"subclass_transaction_amt":"sales"})
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['sales']=df['sales'].astype(float)
    df_sales=df.groupby(['customer_id_hashed','transaction_dt'])['sales'].sum().to_frame().reset_index()
    
    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['trans'].sum().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed",'transaction_dt'],how="outer")

    print(i_counter,df['transaction_dt'].min(),df['transaction_dt'].max(),datetime.datetime.now(),file)
    i_counter+=1
    list_df_transactions_18_months.append(df)

0 2016-06-26 2016-07-02 2019-11-11 19:45:46.290675 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2016-07-02.txt
1 2016-07-03 2016-07-09 2019-11-11 19:46:48.301229 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2016-07-09.txt
2 2016-07-10 2016-07-16 2019-11-11 19:47:50.088198 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2016-07-16.txt
3 2016-07-17 2016-07-23 2019-11-11 19:48:27.282301 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2016-07-23.txt
4 2016-07-24 2016-07-30 2019-11-11 19:49:07.000595 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2016-07-30.txt
5 2016-07-31 2016-08-06 2019-11-11 19:49:48.503444 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2016-08-06.txt
6 2016-08-07 2016-08-13 2019-11-11 19:50:29.890201 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDa

56 2017-07-23 2017-07-29 2019-11-11 20:29:40.341628 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-07-29.txt
57 2017-07-30 2017-08-05 2019-11-11 20:30:28.534913 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-08-05.txt
58 2017-08-06 2017-08-12 2019-11-11 20:31:18.346253 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-08-12.txt
59 2017-08-13 2017-08-19 2019-11-11 20:32:12.106114 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-08-19.txt
60 2017-08-20 2017-08-26 2019-11-11 20:33:00.739684 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-08-26.txt
61 2017-08-27 2017-09-02 2019-11-11 20:33:51.869709 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-09-02.txt
62 2017-09-03 2017-09-09 2019-11-11 20:34:46.134945 /home/jian/BigLots/hist_daily_data_subclasslevel/Media

In [6]:
df_transactions_18_months=pd.concat(list_df_transactions_18_months)
del list_df_transactions_18_months
gc.collect()


0

In [7]:
df_transactions_18_months=df_transactions_18_months[(df_transactions_18_months['transaction_dt']>=str(lapsed_Sunday)) & (df_transactions_18_months['transaction_dt']<=str(lastdate_date))]
df_transactions_18_months.shape

(65383860, 4)

In [8]:
print(df_transactions_18_months['transaction_dt'].min())

min_date_above_str=str(df_transactions_18_months['transaction_dt'].min())

2016-06-26


In [9]:
'''
df=pd.read_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv',dtype=str,usecols=['transaction_date'])
print(df['transaction_date'].min())
2016-06-26
# The same above, no need to read the cumulative sales
'''

"\ndf=pd.read_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv',dtype=str,usecols=['transaction_date'])\nprint(df['transaction_date'].min())\n2016-06-26\n# The same above, no need to read the cumulative sales\n"

In [10]:
df_transactions_18_months.head(2)

,customer_id_hashed,transaction_dt,sales,trans
0,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...,2016-06-30,17.9,1
1,00002d97b5f2e071ede88fa136ed32ca68d8663419767e...,2016-06-27,31.0,1


In [11]:
# 2 out of 2

df_lapsed_sales_ids=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date','location_id','transaction_id','total_transaction_amt'],dtype=str)# .drop_duplicates() # Doesn't go to score at all, so no need to read all columns
df_lapsed_sales_ids=df_lapsed_sales_ids[(df_lapsed_sales_ids['transaction_date']>=str(lapsed_Sunday)) & (df_lapsed_sales_ids['transaction_date']<min_date_above_str)]
print(df_lapsed_sales_ids['transaction_date'].min(),df_lapsed_sales_ids['transaction_date'].max())

df_lapsed_sales_ids=df_lapsed_sales_ids.drop_duplicates()
df_lapsed_sales_ids['total_transaction_amt']=df_lapsed_sales_ids['total_transaction_amt'].astype(float)
df_lapsed_sales_ids=df_lapsed_sales_ids.rename(columns={"transaction_date":"transaction_dt"})

2016-04-03 2016-06-25


In [12]:
df_sales=df_lapsed_sales_ids.groupby(['customer_id_hashed','transaction_dt'])['total_transaction_amt'].sum().to_frame().reset_index().rename(columns={"total_transaction_amt":"sales"})
    
df_trans=df_lapsed_sales_ids[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
df_trans['trans']=1
df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['trans'].sum().to_frame().reset_index()

df_lapsed_sales_ids=pd.merge(df_sales,df_trans,on=["customer_id_hashed",'transaction_dt'],how="outer")

print(df_lapsed_sales_ids['transaction_dt'].max())
print(df_lapsed_sales_ids['transaction_dt'].min())
print(df_lapsed_sales_ids['transaction_dt'].nunique())
print(df_lapsed_sales_ids.shape)

2016-06-25
2016-04-03
84
(153197, 4)


In [13]:
df_transactions_18_months=df_transactions_18_months.append(df_lapsed_sales_ids)
print(df_transactions_18_months['transaction_dt'].max())
print(df_transactions_18_months['transaction_dt'].min())
print(df_transactions_18_months['transaction_dt'].nunique())
print(df_transactions_18_months.shape)

2017-09-30
2016-04-03
545
(65537057, 4)


In [14]:
agg_f={"transaction_dt":"max","sales":"sum","trans":"sum"}
df_sales_by_id=df_transactions_18_months.groupby("customer_id_hashed")['transaction_dt','sales','trans'].agg(agg_f).reset_index()
df_sales_by_id=df_sales_by_id.rename(columns={"transaction_dt":"last_trans_date"})
df_sales_by_id.shape

(15972206, 4)

In [15]:
df_sales_by_id['last_trans_date']=pd.to_datetime(df_sales_by_id['last_trans_date'])
df_sales_by_id['recency']= lastdate_date - df_sales_by_id['last_trans_date'].dt.date
df_sales_by_id['recency']=df_sales_by_id['recency'].apply(lambda x:x.days)
df_sales_by_id['recency']=np.ceil((df_sales_by_id['recency']+1)/30)

In [16]:
df_sales_by_id=df_sales_by_id.sort_values(['trans','recency','sales'],ascending=[0,1,0])
df_sales_by_id.reset_index(drop=True, inplace=True)
df_sales_by_id.reset_index(inplace=True)
df_sales_by_id=df_sales_by_id.rename(columns={'index':'Transindex'})

df_sales_by_id=df_sales_by_id.sort_values(['sales','recency','trans'],ascending=[0,1,0])
df_sales_by_id.reset_index(drop=True, inplace=True)
df_sales_by_id.reset_index(inplace=True)
df_sales_by_id=df_sales_by_id.rename(columns={'index':'Amtindex'})

df_sales_by_id=df_sales_by_id.sort_values(['recency','trans','sales'],ascending=[1,0,0])
df_sales_by_id.reset_index(drop=True, inplace=True)
df_sales_by_id.reset_index(inplace=True)
df_sales_by_id=df_sales_by_id.rename(columns={'index':'recencyindex'})

c_ids=len(df_sales_by_id.index)
c_ids=c_ids//5

df_sales_by_id['Transindex']=np.ceil((df_sales_by_id['Transindex']+1)/c_ids)
df_sales_by_id['Amtindex']=np.ceil((df_sales_by_id['Amtindex']+1)/c_ids)
df_sales_by_id['recencyindex']=np.ceil((df_sales_by_id['recencyindex']+1)/c_ids)

df_sales_by_id['RFM']=df_sales_by_id['recencyindex']*100 + df_sales_by_id['Transindex']*10 + df_sales_by_id['Amtindex']


In [17]:
df_sales_by_id = df_sales_by_id.sort_values(['RFM','recency','trans',
                               'sales'],ascending = [1,1,0,0])

print(df_sales_by_id.shape)

df_sales_by_id.reset_index(drop = True, inplace = True)
df_sales_by_id.reset_index(inplace = True)

df_sales_by_id = df_sales_by_id.rename(columns = {'index':'frmindex'})
c_ids = len(df_sales_by_id.index)//10+1

df_sales_by_id['frmindex'] = np.ceil((df_sales_by_id['frmindex']+1)/c_ids)

print("Done with the RFM scoring, "+str(datetime.datetime.now()))

df_sales_by_id['frmindex']=df_sales_by_id['frmindex'].apply(lambda x: "D"+str(int(float(x))).zfill(2))

(15972206, 9)
Done with the RFM scoring, 2019-11-11 21:37:53.107882


In [18]:
df_H=pd.DataFrame({"frmindex":['D01','D02','D03','D04']})
df_H['HML_Group']="H"

df_M=pd.DataFrame({"frmindex":['D05','D06','D07']})
df_M['HML_Group']="M"

df_L=pd.DataFrame({"frmindex":['D08','D09','D10']})
df_L['HML_Group']="L"

df_HML=df_H.append(df_M).append(df_L)

df_HML.head(2)

,frmindex,HML_Group
0,D01,H
1,D02,H


In [19]:
df_sales_by_id=pd.merge(df_sales_by_id,df_HML,on="frmindex",how="left")
print(df_sales_by_id.shape)

(15972206, 11)


In [20]:
df_master=pd.read_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/BL_all_master_id_email_zip_closet_store_ZipLabel_upto_20191012_JL_2019-10-21',
                       usecols=['customer_id_hashed','cleaned_zip_cd','zip_type'],dtype=str)

print(df_master.shape,df_master['customer_id_hashed'].nunique())

(30732606, 3) 30732606


In [21]:
df_sales_by_id=pd.merge(df_sales_by_id,df_master,on="customer_id_hashed",how="left")

In [22]:
df_sales_by_id.to_csv("./BL_Loyalty_Health_17_18_HML_score_details_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [23]:
df_sales_by_id.head(2)

,frmindex,recencyindex,Amtindex,Transindex,customer_id_hashed,last_trans_date,sales,trans,recency,RFM,HML_Group,cleaned_zip_cd,zip_type
0,D01,1.0,1.0,1.0,a559dc73bfa37290f41696627e867601815ee097a3fd7b...,2017-09-29,21263.32,1083,1.0,111.0,H,71901,trans_S
1,D01,1.0,1.0,1.0,006e60ad4a22343cebb282056a0c65ee7d5061720d8a82...,2017-09-08,22441.37,1040,1.0,111.0,H,71913,trans_P


In [24]:
df_sales_by_id['zip_type']=df_sales_by_id['zip_type'].fillna("T")
df_sales_by_id['zip_type']=df_sales_by_id['zip_type'].replace("trans_P","P").replace("trans_S","S").replace("zips_10","S")

In [25]:
df_sales_by_id=df_sales_by_id[['customer_id_hashed','frmindex','last_trans_date','HML_Group','zip_type']]
gc.collect()

0

In [26]:
df_sales_by_id['Active_Status']=np.where(df_sales_by_id['last_trans_date']>=active_Sunday,"Active (0-12)","Lapsed (13-18)")


In [27]:
df_sales_by_id['last_trans_date'].min()

Timestamp('2016-04-03 00:00:00')

In [28]:
lastdate_date

datetime.date(2017, 9, 30)

In [29]:
Beginning_18_months_ago

'2016-04-03'

In [30]:
df_sales_by_id.head(2)

,customer_id_hashed,frmindex,last_trans_date,HML_Group,zip_type,Active_Status
0,a559dc73bfa37290f41696627e867601815ee097a3fd7b...,D01,2017-09-29,H,S,Active (0-12)
1,006e60ad4a22343cebb282056a0c65ee7d5061720d8a82...,D01,2017-09-08,H,P,Active (0-12)


In [31]:
df_sales_by_id['last_trans_date'].min()

Timestamp('2016-04-03 00:00:00')

In [32]:
lastdate_date

datetime.date(2017, 9, 30)

In [33]:
# Lapsed 19 - 48


df_lapsed_sales_ids=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date','location_id','transaction_id','total_transaction_amt'],dtype=str)# .drop_duplicates() # Doesn't go to score at all, so no need to read all columns
df_lapsed_sales_ids=df_lapsed_sales_ids[(df_lapsed_sales_ids['transaction_date']<str(lapsed_Sunday))]
print(df_lapsed_sales_ids['transaction_date'].min(),df_lapsed_sales_ids['transaction_date'].max())

df_lapsed_sales_ids=df_lapsed_sales_ids.drop_duplicates()
df_lapsed_sales_ids['total_transaction_amt']=df_lapsed_sales_ids['total_transaction_amt'].astype(float)
df_lapsed_sales_ids=df_lapsed_sales_ids.rename(columns={"transaction_date":"transaction_dt"})

print("date 48 months earlier",lastdate_date-datetime.timedelta(days=365*4))

2014-08-26 2016-04-02
date 48 months earlier 2013-10-01


In [34]:
df_sales=df_lapsed_sales_ids.groupby(['customer_id_hashed','transaction_dt'])['total_transaction_amt'].sum().to_frame().reset_index().rename(columns={"total_transaction_amt":"sales"})
    
df_trans=df_lapsed_sales_ids[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
df_trans['trans']=1
df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['trans'].sum().to_frame().reset_index()

df_lapsed_sales_ids=pd.merge(df_sales,df_trans,on=["customer_id_hashed",'transaction_dt'],how="outer")

print(df_lapsed_sales_ids['transaction_dt'].max())
print(df_lapsed_sales_ids['transaction_dt'].min())
print(df_lapsed_sales_ids['transaction_dt'].nunique())
print(df_lapsed_sales_ids.shape)

2016-04-02
2014-08-26
584
(974324, 4)


In [35]:
df_lapsed_sales_ids=df_lapsed_sales_ids[~df_lapsed_sales_ids['customer_id_hashed'].isin(df_sales_by_id['customer_id_hashed'].tolist())]
df_lapsed_sales_ids['Active_Status']="Lapsed (19-48)"
df_lapsed_sales_ids.shape

(889162, 5)

In [36]:
df_sales_by_id=df_sales_by_id.append(df_lapsed_sales_ids)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [37]:
del df_lapsed_sales_ids
df_sales_by_id.to_csv("./BL_Loyalty_Health_2017_2018_all_ids_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
gc.collect()

0

# Sale in 1 Year

In [38]:
historical_item_files=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
historical_item_files=[x for x in historical_item_files if x.split("/MediaStormDailySalesHistory")[1][:8]<="20181007"]
historical_item_files.sort()

print(min([x.split("/MediaStormDailySalesHistory")[1][:8] for x in historical_item_files]))

historical_item_files

20180811


['/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180811.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180818.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180825.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180901.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180908.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180915.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180922.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt']

In [39]:
historical_subclass_files=glob.glob("/home/jian/BigLots/hist_daily_data_subclasslevel/*.txt")

historical_subclass_files=[x for x in historical_subclass_files if x.split("/MediaStormDailySales_week_ending_")[1][:10]>="2017-10-01"]
historical_subclass_files=[x for x in historical_subclass_files if x.split("/MediaStormDailySales_week_ending_")[1][:10]<="2018-08-11"]
historical_subclass_files.sort()
historical_subclass_files

['/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-07.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-14.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-21.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-28.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-04.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-11.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-18.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-25.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-12-02.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-12

In [40]:
daily_POS_files_2018=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
daily_POS_files_2018=[x for x in daily_POS_files_2018 if "daily" in x.lower()]
daily_POS_files_2018=[x for x in daily_POS_files_2018 if ".txt" in x.lower()]
daily_POS_files_2018=[x for x in daily_POS_files_2018 if x.split("MediaStorm_")[1][:10]<="2018-08-11"]

daily_POS_files_2018.sort()

daily_POS_files_2018

['/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-16/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-23/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-30/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-07/MediaStormDailySales.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-14/MediaStormDailySales20180717-113630-767.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-21/MediaStormDailySales20180724-113327-741.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-28/MediaStormDailySales20180731-111804-489.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-08-04/MediaStormDailySales20180807-111637-702.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-08-11/MediaStormDailySales20180814-121633-120.txt']

In [41]:
list_POS_item_in_1_year=historical_subclass_files+daily_POS_files_2018+historical_item_files
print(len(list_POS_item_in_1_year))

list_POS_item_in_1_year

54


['/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-07.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-14.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-21.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-28.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-04.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-11.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-18.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-25.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-12-02.txt',
 '/home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-12

In [42]:
df_sales_in_1_year=pd.DataFrame()
i_counter=0
for file in list_POS_item_in_1_year:
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
        df=df.rename(columns={"item_transaction_amt":"sales"})
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','subclass_transaction_amt'])
        df=df.rename(columns={"subclass_transaction_amt":"sales"})
        
    df=df[pd.notnull(df['customer_id_hashed'])]
    ###
    df=df[df['transaction_dt']>="2017-10-01"]
    df=df[df['transaction_dt']<="2018-09-30"]
    ###
    df['sales']=df['sales'].astype(float)
    df_sales=df.groupby(['customer_id_hashed','transaction_dt'])['sales'].sum().to_frame().reset_index()
    
    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['trans'].sum().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed",'transaction_dt'],how="outer")

    print(i_counter,df['transaction_dt'].min(),df['transaction_dt'].max(),datetime.datetime.now(),file)
    i_counter+=1
    df_sales_in_1_year=df_sales_in_1_year.append(df)


0 2017-10-01 2017-10-07 2019-11-11 21:54:25.171389 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-07.txt
1 2017-10-08 2017-10-14 2019-11-11 21:55:07.225928 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-14.txt
2 2017-10-15 2017-10-21 2019-11-11 21:55:54.552339 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-21.txt
3 2017-10-22 2017-10-28 2019-11-11 21:56:44.006205 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-10-28.txt
4 2017-10-29 2017-11-04 2019-11-11 21:57:36.153691 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-04.txt
5 2017-11-05 2017-11-11 2019-11-11 21:58:35.206661 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDailySales_week_ending_2017-11-11.txt
6 2017-11-12 2017-11-18 2019-11-11 21:59:43.433346 /home/jian/BigLots/hist_daily_data_subclasslevel/MediaStormDa

In [43]:
gc.collect()
print(datetime.datetime.now())
agg_f={"transaction_dt":"max","sales":"sum","trans":"sum"}
df_sales_in_1_year=df_sales_in_1_year.groupby("customer_id_hashed")['transaction_dt','sales','trans'].agg(agg_f).reset_index()
df_sales_in_1_year=df_sales_in_1_year.rename(columns={"transaction_dt":"last_trans_date_in_1_year"})
df_sales_in_1_year=df_sales_in_1_year.rename(columns={"sales":"sales_in_1_year"})
df_sales_in_1_year=df_sales_in_1_year.rename(columns={"trans":"trans_in_1_year"})
print(datetime.datetime.now())

df_sales_in_1_year.shape

2019-11-11 22:55:26.830195
2019-11-11 23:52:53.134329


(16283224, 4)

In [44]:
df_sales_in_1_year.head(2)

,customer_id_hashed,last_trans_date_in_1_year,sales_in_1_year,trans_in_1_year
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-09-22,58.70,1
1,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,2017-11-10,39.99,1


In [45]:
df_sales_in_1_year['customer_id_hashed'].nunique()

16283224

In [46]:
df_sales_by_id.shape

(16861368, 9)

In [47]:
print(df_sales_by_id.shape)
df_sales_by_id=pd.merge(df_sales_by_id,df_sales_in_1_year,on="customer_id_hashed",how="left")
print(df_sales_by_id.shape)
df_sales_by_id['sales_in_1_year']=df_sales_by_id['sales_in_1_year'].fillna(0)
df_sales_by_id['trans_in_1_year']=df_sales_by_id['trans_in_1_year'].fillna(0)


(16861368, 9)
(16861368, 12)


In [48]:
del df_sales_in_1_year
gc.collect()

0

In [49]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Loyalty_Health'

In [50]:
df_sales_by_id.to_csv("./BL_2017Audience_Performance_in_detail_18_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [51]:
df_sales_by_id['shoppers_in_1_year']=np.where(pd.isnull(df_sales_by_id['last_trans_date_in_1_year']),0,1)
df_sales_by_id['zip_type']=df_sales_by_id['zip_type'].fillna("T")
df_sales_by_id['HML_Group']=df_sales_by_id['HML_Group'].fillna("nan")

In [62]:
df_sales_by_id.shape

(16861368, 13)

In [64]:
df_sales_by_id['customer_id_hashed'].nunique()

16861368

In [52]:
df_sales_by_id['Active_Status'].unique()

array(['Active (0-12)', 'Lapsed (13-18)', 'Lapsed (19-48)'], dtype=object)

In [53]:
df_sales_by_id['HML_Group'].unique()

array(['H', 'M', 'L', 'nan'], dtype=object)

In [54]:
df_sales_by_id['zip_type'].unique()

array(['S', 'P', 'T'], dtype=object)

In [55]:
df_sales_by_id.head(2)

,Active_Status,HML_Group,customer_id_hashed,frmindex,last_trans_date,sales,trans,transaction_dt,zip_type,last_trans_date_in_1_year,sales_in_1_year,trans_in_1_year,shoppers_in_1_year
0,Active (0-12),H,a559dc73bfa37290f41696627e867601815ee097a3fd7b...,D01,2017-09-29,NaN,NaN,NaN,S,2018-08-15,43.05,11.0,1
1,Active (0-12),H,006e60ad4a22343cebb282056a0c65ee7d5061720d8a82...,D01,2017-09-08,NaN,NaN,NaN,P,2018-02-22,73.99,3.0,1


In [56]:
agg_f={"customer_id_hashed":"count","shoppers_in_1_year":"sum","sales_in_1_year":"sum","trans_in_1_year":"sum"}

df_output=df_sales_by_id.groupby(["Active_Status",'HML_Group','zip_type'])['customer_id_hashed','shoppers_in_1_year','sales_in_1_year','trans_in_1_year'].agg(agg_f).reset_index()
df_output.shape

(13, 7)

In [57]:
df_output.to_csv("./BL_2017Audience_Performance_in_summary_18_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [58]:
df_output

,Active_Status,HML_Group,zip_type,customer_id_hashed,shoppers_in_1_year,sales_in_1_year,trans_in_1_year
0,Active (0-12),H,P,4281974,3470531,8.602852e+08,26834112.0
1,Active (0-12),H,S,1525516,1182417,2.781150e+08,7871602.0
2,Active (0-12),H,T,581394,419983,9.547690e+07,2635810.0
3,Active (0-12),L,P,1901192,756093,8.686049e+07,2097977.0
4,Active (0-12),L,S,799446,301565,3.535424e+07,782106.0
5,Active (0-12),L,T,353995,116559,1.373815e+07,294347.0
6,Active (0-12),M,P,3109076,1898952,2.821960e+08,7378658.0
7,Active (0-12),M,S,1208864,706565,1.059036e+08,2536376.0
8,Active (0-12),M,T,473723,252827,3.763555e+07,875247.0
9,Lapsed (13-18),L,P,1092252,353338,3.967417e+07,944535.0


In [65]:
df_output[df_output['Active_Status']!="Lapsed (19-48)"]['customer_id_hashed'].sum()

15972206

In [59]:
df_output['customer_id_hashed'].sum()

16861368

In [66]:
889162/15972206

0.055669329584153875

In [60]:
df_sales_by_id[(df_sales_by_id['Active_Status']=="Lapsed (13-18)") & (df_sales_by_id['HML_Group']=="nan")]

,Active_Status,HML_Group,customer_id_hashed,frmindex,last_trans_date,sales,trans,transaction_dt,zip_type,last_trans_date_in_1_year,sales_in_1_year,trans_in_1_year,shoppers_in_1_year


In [61]:
print(datetime.datetime.now())

2019-11-12 00:06:30.340017
